In [0]:
import tensorflow as tf

In [2]:
# Tensorboard in Colab environment.
!pip install tensorboardcolab
from tensorboardcolab import *
tbc = TensorBoardColab()

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://e61f6b25.ngrok.io


In [0]:
# Provide the ground truth last word as input to the RNN, not the last word you predicted. This is common practice.
# do this only in the train phase not the validation right?

# what to do with ''' in sentences ???

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
base_folder = '/gdrive/My Drive/Colab Notebooks/NLU/data/'

sentence_length = 30

In [0]:
def preprocess_sentences(sentence):
  tokens = sentence.split(" ")[:-1]
  sentece_length_no_bos_eos = sentence_length - 2
  tokenized_sentence = ['<bos>'] + tokens[:sentece_length_no_bos_eos] + ['<pad>'] * (sentece_length_no_bos_eos - len(tokens)) + ['<eos>']
  return tokenized_sentence

def get_sentences_from_file(file_name, maximum_number_of_sentences=-1):
  fp = open(base_folder + file_name)
  irofile = iter(fp)
  
  sentences = []
  for line in irofile:
    if maximum_number_of_sentences <= 0:
      break
    maximum_number_of_sentences -= 1
    sentences.append(preprocess_sentences(line))
    
  fp.close()
  return sentences

In [0]:
sentences_train = get_sentences_from_file('sentences.train', maximum_number_of_sentences=10000)

## create LSTM

In [0]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

log_dir = "/tmp/tensorflow/mnist_cnn/logs"
tf.app.flags.DEFINE_string("log_dir", log_dir, "Summaries log directory")

In [0]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor for TensorBoard visualizations."""
  mean = tf.reduce_mean(var)
  tf.summary.scalar('mean', mean)
  with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
  tf.summary.scalar('stddev', stddev)
  tf.summary.scalar('max', tf.reduce_max(var))
  tf.summary.scalar('min', tf.reduce_min(var))
  tf.summary.histogram('histogram', var)

In [0]:
import numpy as np

def get_vocabulary_size(sentences):
  vocabulary = set()
  for word in np.array(sentences).flatten():
    vocabulary.add(word)
    
  vocabulary.add('<bos>')
  vocabulary.add('<eos>')
  vocabulary.add('<pad>')
  vocabulary.add('<unk>')
  
  word2number_mapping = dict([(y,x) for x,y in enumerate(sorted(vocabulary))])
  number2word_mapping = dict([(x,y) for x,y in enumerate(sorted(vocabulary))])
  
  return word2number_mapping, number2word_mapping, len(vocabulary)


word2number_mapping, number2word_mapping, vocabulary_size = get_vocabulary_size(sentences_train)

In [0]:
def sentences2vec(sentences, word2number_mapping):
  sentence_vectors = []
  for sentence in sentences:
    sentence_vectors.append([word2number_mapping[word] for word in sentence])
  return sentence_vectors

sentence_vectors_train = sentences2vec(sentences_train, word2number_mapping)

In [0]:
embedding_size = 100
hidden_state_size = 512

In [0]:
def compute_equations(weights_i, weights_f, weights_o, weights_g, concatenated_input, c):
  concatenated_input = tf.reshape(concatenated_input, [-1, 1])
  i = tf.sigmoid(tf.matmul(weights_i, concatenated_input, name='matmul_i'), name='calculation_i')
  f = tf.sigmoid(tf.matmul(weights_f, concatenated_input, name='matmul_f'), name='calculation_f')
  o = tf.sigmoid(tf.matmul(weights_o, concatenated_input, name='matmul_o'), name='calculation_o')
  g = tf.tanh(tf.matmul(weights_g, concatenated_input, name='matmul_g'), name='calculation_g')
  
  c = tf.multiply(f, c) + tf.multiply(i, g)
  h = np.multiply(o, tf.tanh(c))
  
  return i, f, o, g, c, h
  
def weight_variable(shape, name):
  # todo , initializer=tf.contrib.layers.xavier_initializer()
  return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name=name)

def LSTM():
  input_sentence = tf.placeholder(tf.int32, shape=[sentence_length], name='input_sentence')
  output_sentence = tf.placeholder(tf.int32, shape=[sentence_length - 1], name='output_sentence')
  
  input_embeddings = weight_variable([vocabulary_size, embedding_size], name='input_embeddings')
  
  sentence_embedding = tf.nn.embedding_lookup(input_embeddings, input_sentence, name='sentence_embedding')
  
#   print(sentence_embedding.eval())
   
  # compute functions i, f, o, g as in https://arxiv.org/pdf/1409.2329.pdf
  weights_i = weight_variable([hidden_state_size, hidden_state_size + embedding_size + 1], name='weights_i')
#   variable_summaries(weights_i)
  weights_f = weight_variable([hidden_state_size, hidden_state_size + embedding_size + 1], name='weights_f')
#   variable_summaries(weights_f)
  weights_o = weight_variable([hidden_state_size, hidden_state_size + embedding_size + 1], name='weights_o')
#   variable_summaries(weights_o)
  weights_g = weight_variable([hidden_state_size, hidden_state_size + embedding_size + 1], name='weights_g')
#   variable_summaries(weights_g)
  
  weights_output = weight_variable([vocabulary_size, hidden_state_size], name='weights_output')
  
  hidden_state = tf.zeros([hidden_state_size], tf.float32)
#   tf.summary.histogram('hidden_state', hidden_state)
  
  c = tf.zeros([hidden_state_size, 1], tf.float32)
#   tf.summary.histogram('c', c)
  
  outputs = []
  for i in range(sentence_length - 1):
    input_t = sentence_embedding[i]
  
    concatenated_input = tf.concat([tf.reshape(hidden_state, [-1]), tf.cast(input_t, tf.float32), [1.0]], 0)
    
    i, f, o, g, c, hidden_state = compute_equations(weights_i, weights_f, weights_o, weights_g, concatenated_input, c)
    
    output_t = tf.reshape(tf.matmul(weights_output, hidden_state, name='matmul_output'), [-1])
    
    outputs.append(output_t)
    
  cross_entropy_loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=outputs, labels=output_sentence))
  
  output_probabilities = tf.nn.softmax(outputs)
  
  return input_sentence, output_sentence, outputs, cross_entropy_loss

In [0]:
# counter = 0

# def senteces2vec_to_categorical(sentence_vector):
#   sentence_to_categorical = np.zeros((sentence_length - 1, vocabulary_size))

#   for i in range(sentence_length - 1):
#     sentence_to_categorical[i, sentence_vector[i]] = 1
      
#   return sentence_to_categorical

def get_sentence(sentence_vectors_train, pos):
  sentence_vector = sentence_vectors_train[pos]
  #labels = senteces2vec_to_categorical(sentence_vector)
  
  return sentence_vector, sentence_vector[1:]

In [92]:
sentence_vector, labels = get_sentence(sentence_vectors_train, 0)
np.array(sentence_vector)

array([ 151,  158, 5156,   77, 7035, 4715,  597, 5165, 4099, 6923,   94,
       6917, 3812, 7035, 6064, 6539, 4839,  597,  110,  153,  153,  153,
        153,  153,  153,  153,  153,  153,  153,  152])

In [98]:
with tf.Session() as session:
  input_sentence, output_sentence, output_probs, loss = LSTM()
  
  optimizer = tf.train.AdamOptimizer().minimize(loss)
#   optimizer = tf.train.AdamOptimizer()
#   gvs = optimizer.compute_gradients(loss)
#   capped_gvs = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gvs]
#   train_op = optimizer.apply_gradients(capped_gvs)
  
  session.run(tf.global_variables_initializer())
  
  
  n_epochs = 10
  for epoch in range(n_epochs):
    
    loss_epoch = 0
    
    test_size = 100
    for pos in range(test_size):
      sentence_vector, labels = get_sentence(sentence_vectors_train, pos)

      loss_sample, _ = session.run([loss, optimizer], feed_dict={input_sentence: sentence_vector, output_sentence: labels})
    
      loss_epoch += loss_sample
      
    loss_epoch /= test_size
    print(f'epoch: {epoch:3d} loss: {loss_epoch:.4f}')

epoch:   0 loss: 3.7365
epoch:   1 loss: 2.4521
epoch:   2 loss: 2.2366
epoch:   3 loss: 2.1029
epoch:   4 loss: 2.0013
epoch:   5 loss: 1.8753
epoch:   6 loss: 1.7603
epoch:   7 loss: 1.6652
epoch:   8 loss: 1.5369
epoch:   9 loss: 1.3941


In [0]:
# saver = tf.train.Saver() 
# saver.save(sess, 'filename.chkp')

# Then you'll be able to access the model:

# sess = tf.Session()
# saver = tf.train.Saver()
# saver.restore(sess, 'filename.chkp')

In [88]:
k = 1
for i in b[0]:
  prediction = np.argmax(i)
  print(f'Prediction: {number2word_mapping[prediction]:10s} correct word: {number2word_mapping[sentence_vector[k]]}')
  k += 1

Prediction: ``         correct word: ``
Prediction: i          correct word: i
Prediction: 've        correct word: 've
Prediction: never      correct word: never
Prediction: had        correct word: had
Prediction: any        correct word: any
Prediction: ice-cream  correct word: ice-cream
Prediction: for        correct word: for
Prediction: myself     correct word: myself
Prediction: ,          correct word: ,
Prediction: my         correct word: my
Prediction: father     correct word: father
Prediction: never      correct word: never
Prediction: let        correct word: let
Prediction: me         correct word: me
Prediction: have       correct word: have
Prediction: any        correct word: any
Prediction: .          correct word: .
Prediction: <pad>      correct word: <pad>
Prediction: <pad>      correct word: <pad>
Prediction: <pad>      correct word: <pad>
Prediction: <pad>      correct word: <pad>
Prediction: <pad>      correct word: <pad>
Prediction: <pad>      correct word: <p

In [49]:
with tf.Session() as session:
  a = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=np.array([[1,2,3],[1,2,3]], dtype=np.float32), labels=[0,2])
  print(a.eval())

[2.407606   0.40760598]


In [54]:
sentence_vector, labels = get_sentence(sentence_vectors_train)
sentence_vector[1:]

[158,
 5156,
 77,
 7035,
 4715,
 597,
 5165,
 4099,
 6923,
 94,
 6917,
 3812,
 7035,
 6064,
 6539,
 4839,
 597,
 110,
 153,
 153,
 153,
 153,
 153,
 153,
 153,
 153,
 153,
 153,
 152]

In [0]:
# Create the session
tf.reset_default_graph()
sess.close()
sess = tf.InteractiveSession()

# Tensorboard
train_writer = tbc.get_deep_writers("single_layer_model/train")
train_writer.add_graph(sess.graph)
valid_writer = tbc.get_deep_writers("single_layer_model/valid")
valid_writer.add_graph(sess.graph)

# Initialize all variables
sess.run(tf.global_variables_initializer())

# To be able to see something in tensorboard, we must merge summaries to one common operation.
# Whenever we want to write summaries, we must request this operation from the graph.
# Note: creating the file writers should happen after the session was launched.
summaries_merged = tf.summary.merge_all()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
run(sess)
train_writer.flush()
valid_writer.flush()

InvalidArgumentError: ignored

In [0]:
t1 = [1,2,3]
t2 = [2,3,4]


t3 = tf.multiply(t1, t2) + tf.multiply(t1, t2) 
t3.eval()

array([ 4, 12, 24], dtype=int32)

In [0]:

session.close()

In [0]:
sentence_to_categorical.shape

NameError: ignored